In [2]:
from typing import List, Optional, Tuple

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

import sys
sys.path.append('../training')
from networks.wavenet import WaveNet

## Data preparing

In [17]:
batch_size = 4
time_frame = 200
lin_out = torch.randn([batch_size, 128, time_frame])
tv_timbre = torch.randn([batch_size, 192, time_frame])
hubert_temp = torch.randn([batch_size, 256, time_frame//2])
print(lin_out.shape, tv_timbre.shape, hubert_temp.shape)

cqt = torch.randn([batch_size, 191, time_frame])
pitch = torch.randn([batch_size, time_frame])
p_amp = torch.randn([batch_size, time_frame])
ap_amp = torch.randn([batch_size, time_frame])
print(cqt.shape, pitch.shape, p_amp.shape, ap_amp.shape)

torch.Size([4, 128, 200]) torch.Size([4, 192, 200]) torch.Size([4, 256, 100])
torch.Size([4, 191, 200]) torch.Size([4, 200]) torch.Size([4, 200]) torch.Size([4, 200])


In [6]:
hubert_dir = "/disk2/vctk/modified/hubert_soft/p225/p225_001.emb"
hubert_emb = torch.load(hubert_dir)
hubert_emb.shape

torch.Size([102, 256])

## Frame-level Synthesizer

In [18]:
class ConvGLU(nn.Module):
    """Dropout - Conv1d - GLU and conditional layer normalization.
    """
    def __init__(self,
                 channels: int,
                 kernels: int,
                 dilations: int,
                 dropout: float,
                 cond: Optional[int] = None):
        """Initializer.
        Args:
            channels: size of the input channels.
            kernels: size of the convolutional kernels.
            dilations: dilation rate of the convolution.
            dropout: dropout rate.
            cond: size of the condition channels, if provided.
        """
        super().__init__()
        self.conv = nn.Sequential(
            nn.Dropout(dropout),
            nn.Conv1d(channels, channels * 2, kernels, dilation=dilations,
                      padding=(kernels - 1) * dilations // 2),
            nn.GLU(dim=1))
        
        self.cond = cond
        if cond is not None:
            self.cond = nn.Conv1d(cond, channels * 2, 1)

    def forward(self, inputs: torch.Tensor, cond: Optional[torch.Tensor] = None):
        """Transform the inputs with given conditions.
        Args:
            inputs: [torch.float32; [B, channels, T]], input channels.
            cond: [torch.float32; [B, cond, T]], if provided.
        Returns:
            [torch.float32; [B, channels, T]], transformed.
        """
        # [B, channels, T]
        x = inputs + self.conv(inputs)
        if cond is not None:
            assert self.cond is not None, 'condition module does not exists'
            # [B, channels, T]
            x = F.instance_norm(x, use_input_stats=True)
            # [B, channels, T]
            weight, bias = self.cond(cond).chunk(2, dim=1)
            # [B, channels, T]
            x = x * weight + bias
        return x


class CondSequential(nn.Module):
    """Sequential pass with conditional inputs.
    """
    def __init__(self, modules: List[nn.Module]):
        """Initializer.
        Args:
            modules: list of torch modules.
        """
        super().__init__()
        self.lists = nn.ModuleList(modules)
    
    def forward(self, inputs: torch.Tensor, *args, **kwargs) -> torch.Tensor:
        """Pass the inputs to modules.
        Args:
            inputs: arbitary input tensors.
            args, kwargs: positional, keyword arguments.
        Returns:
            output tensor.
        """
        x = inputs
        for module in self.lists:
            x = module.forward(x, *args, **kwargs)
        return x


class FrameLevelSynthesizer(nn.Module):
    """Frame-level synthesizer.
    """
    def __init__(self,
                 channels: int,
                 embed: int,
                 kernels: int,
                 dilations: List[int],
                 blocks: int,
                 leak: float,
                 dropout: float):
        """Initializer.
        Args:
            channels: size of the input channels.
            embed: size of the time-varying timber embeddings.
            kernels: size of the convolutional kernels.
            dilations: dilation rates.
            blocks: the number of the 1x1 ConvGLU blocks after dilated ConvGLU.
            leak: negative slope of the leaky relu.
            dropout: dropout rates.
        """
        super().__init__()
        # channels=1024
        # unknown `leak`, `dropout`
        self.preconv = nn.Sequential(
            nn.Conv1d(channels, channels, 1),
            nn.LeakyReLU(leak),
            nn.Dropout(dropout))
        self.upsample = nn.Sequential(
            nn.Conv1d(channels, channels, 5, 1, 2),
            nn.ReLU(),
            nn.InstanceNorm1d(channels),
            nn.ConvTranspose1d(channels, channels, 4, 2, 1),
            nn.Conv1d(channels, channels, 5, 1, 2),
            nn.ReLU(),
            nn.InstanceNorm1d(channels),
            nn.Conv1d(channels, channels, 5, 1, 2),
            nn.ReLU(),
            nn.InstanceNorm1d(channels),
            )
        # kernels=3, dilations=[1, 3, 9, 27, 1, 3, 9, 27], blocks=2
        self.convglu = CondSequential(
            [
                ConvGLU(channels, kernels, dilation, dropout, cond=embed)
                for dilation in dilations]
            + [
                ConvGLU(channels, 1, 1, dropout, cond=embed)
                for _ in range(blocks)])

        self.proj = nn.Conv1d(channels, channels, 1)

    def forward(self, inputs: torch.Tensor, timber: torch.Tensor) -> torch.Tensor:
        """Synthesize in frame-level.
        Args:
            inputs: [torch.float32; [B, channels, T]], input features.
            timber: [torch.float32; [B, embed, T]], time-varying timber embeddings.
        Returns;
            [torch.float32; [B, channels, T]], outputs.
        """
        # [B, channels, T]
        x = self.preconv(inputs)
        # [B, channels, 2T]
        x = self.upsample(x)
        # [B, channels, 2T] 
        x = self.convglu(x, timber)
        # [B, channels, 2T]
        return self.proj(x)

In [19]:
fsynthesizer = FrameLevelSynthesizer(256, 192, 3, [1, 3, 9, 27, 1, 3, 9, 27], 2, 0.01, 0.1)

In [20]:
fcondition = fsynthesizer(hubert_temp, tv_timbre)
fcondition.shape

torch.Size([4, 256, 200])

In [36]:
upsampler = nn.Upsample(scale_factor=320, mode='linear')
new_pitch = upsampler(pitch[:,None]).squeeze(dim=1)
pitch[:,None].shape, new_pitch.shape

(torch.Size([4, 1, 200]), torch.Size([4, 64000]))